# Pipeline: vLLM Text Generation с Батчами

Этот notebook демонстрирует как использовать vLLM для эффективной генерации текста с поддержкой батчей.

## Особенности:
- **Высокая производительность**: vLLM использует PagedAttention для эффективного управления памятью
- **Батчинг**: Обработка множества запросов одновременно
- **Continuous batching**: Динамическое добавление запросов в батч
- **Поддержка различных моделей**: LLaMA, Mistral, Qwen, Phi и другие
- **Sampling параметры**: Temperature, top-p, top-k, beam search

## Установка

In [ ]:
# Установка vLLM
!pip install vllm -q
!pip install transformers accelerate -q

## 1. Импорты и Конфигурация

In [ ]:
import pandas as pd
import numpy as np
from typing import List, Dict, Optional, Union
from tqdm.auto import tqdm
import time
import json

from vllm import LLM, SamplingParams
from vllm.outputs import RequestOutput

# Конфигурация
MODEL_NAME = "microsoft/phi-2"  # Можно заменить на: "mistralai/Mistral-7B-v0.1", "meta-llama/Llama-2-7b-hf", "Qwen/Qwen-7B"
MAX_MODEL_LEN = 2048  # Максимальная длина контекста
TENSOR_PARALLEL_SIZE = 1  # Количество GPU для тензорного параллелизма
GPU_MEMORY_UTILIZATION = 0.9  # Процент использования GPU памяти

# Параметры генерации по умолчанию
DEFAULT_TEMPERATURE = 0.7
DEFAULT_TOP_P = 0.9
DEFAULT_TOP_K = 50
DEFAULT_MAX_TOKENS = 256

## 2. Инициализация vLLM модели

In [ ]:
def initialize_vllm_model(
    model_name: str = MODEL_NAME,
    max_model_len: int = MAX_MODEL_LEN,
    tensor_parallel_size: int = TENSOR_PARALLEL_SIZE,
    gpu_memory_utilization: float = GPU_MEMORY_UTILIZATION,
    trust_remote_code: bool = True
) -> LLM:
    """
    Инициализирует vLLM модель
    
    Args:
        model_name: Название модели из HuggingFace
        max_model_len: Максимальная длина контекста
        tensor_parallel_size: Количество GPU
        gpu_memory_utilization: Процент использования GPU памяти (0.0-1.0)
        trust_remote_code: Доверять ли remote code
    
    Returns:
        Инициализированная LLM модель
    """
    print(f"Загрузка модели {model_name}...")
    
    llm = LLM(
        model=model_name,
        max_model_len=max_model_len,
        tensor_parallel_size=tensor_parallel_size,
        gpu_memory_utilization=gpu_memory_utilization,
        trust_remote_code=trust_remote_code,
    )
    
    print(f"Модель {model_name} успешно загружена!")
    return llm

# Инициализация модели
llm = initialize_vllm_model()

## 3. Создание Sampling Parameters

In [ ]:
def create_sampling_params(
    temperature: float = DEFAULT_TEMPERATURE,
    top_p: float = DEFAULT_TOP_P,
    top_k: int = DEFAULT_TOP_K,
    max_tokens: int = DEFAULT_MAX_TOKENS,
    n: int = 1,
    stop: Optional[List[str]] = None,
    presence_penalty: float = 0.0,
    frequency_penalty: float = 0.0,
    repetition_penalty: float = 1.0,
    use_beam_search: bool = False,
    best_of: Optional[int] = None,
) -> SamplingParams:
    """
    Создает параметры для сэмплирования
    
    Args:
        temperature: Температура сэмплирования (выше = более случайный вывод)
        top_p: Nucleus sampling (рассматриваются токены с суммарной вероятностью top_p)
        top_k: Top-k sampling (рассматриваются top_k наиболее вероятных токенов)
        max_tokens: Максимальное количество генерируемых токенов
        n: Количество генераций на один промпт
        stop: Список стоп-последовательностей
        presence_penalty: Штраф за присутствие токенов (-2.0 to 2.0)
        frequency_penalty: Штраф за частоту токенов (-2.0 to 2.0)
        repetition_penalty: Штраф за повторение (1.0 = нет штрафа)
        use_beam_search: Использовать ли beam search
        best_of: Количество кандидатов для выбора лучшего (для beam search)
    
    Returns:
        SamplingParams объект
    """
    return SamplingParams(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        max_tokens=max_tokens,
        n=n,
        stop=stop,
        presence_penalty=presence_penalty,
        frequency_penalty=frequency_penalty,
        repetition_penalty=repetition_penalty,
        use_beam_search=use_beam_search,
        best_of=best_of,
    )

# Примеры различных конфигураций
sampling_params_creative = create_sampling_params(
    temperature=1.0,
    top_p=0.95,
    max_tokens=512
)

sampling_params_precise = create_sampling_params(
    temperature=0.1,
    top_p=0.9,
    max_tokens=256
)

sampling_params_beam = create_sampling_params(
    temperature=0.0,
    use_beam_search=True,
    best_of=5,
    n=1,
    max_tokens=256
)

print("Sampling parameters созданы!")

## 4. Простая генерация (Single Prompt)

In [ ]:
def generate_single(
    llm: LLM,
    prompt: str,
    sampling_params: Optional[SamplingParams] = None
) -> str:
    """
    Генерирует ответ на один промпт
    
    Args:
        llm: vLLM модель
        prompt: Текст промпта
        sampling_params: Параметры генерации
    
    Returns:
        Сгенерированный текст
    """
    if sampling_params is None:
        sampling_params = create_sampling_params()
    
    outputs = llm.generate([prompt], sampling_params)
    return outputs[0].outputs[0].text

# Пример использования
test_prompt = "Explain quantum computing in simple terms:"
response = generate_single(llm, test_prompt, sampling_params_precise)
print(f"Prompt: {test_prompt}")
print(f"Response: {response}")

## 5. Батч-генерация (Multiple Prompts)

In [ ]:
def generate_batch(
    llm: LLM,
    prompts: List[str],
    sampling_params: Optional[SamplingParams] = None,
    show_progress: bool = True
) -> List[str]:
    """
    Генерирует ответы на батч промптов
    
    Args:
        llm: vLLM модель
        prompts: Список промптов
        sampling_params: Параметры генерации
        show_progress: Показывать ли прогресс
    
    Returns:
        Список сгенерированных текстов
    """
    if sampling_params is None:
        sampling_params = create_sampling_params()
    
    start_time = time.time()
    
    if show_progress:
        print(f"Генерация для {len(prompts)} промптов...")
    
    # vLLM автоматически обрабатывает батчи эффективно
    outputs = llm.generate(prompts, sampling_params)
    
    # Извлечение текстов
    generated_texts = [output.outputs[0].text for output in outputs]
    
    elapsed_time = time.time() - start_time
    
    if show_progress:
        print(f"Генерация завершена за {elapsed_time:.2f}s")
        print(f"Скорость: {len(prompts)/elapsed_time:.2f} промптов/сек")
    
    return generated_texts

# Пример использования
test_prompts = [
    "What is machine learning?",
    "Explain neural networks.",
    "What is deep learning?",
    "Define artificial intelligence.",
    "What are transformers in NLP?"
]

batch_responses = generate_batch(llm, test_prompts, sampling_params_precise)

for prompt, response in zip(test_prompts, batch_responses):
    print(f"\nQ: {prompt}")
    print(f"A: {response[:100]}...")  # Первые 100 символов

## 6. Батч-генерация с разными параметрами

In [ ]:
def generate_batch_with_params(
    llm: LLM,
    prompts: List[str],
    sampling_params_list: Optional[List[SamplingParams]] = None,
    show_progress: bool = True
) -> List[str]:
    """
    Генерирует ответы с индивидуальными параметрами для каждого промпта
    
    Args:
        llm: vLLM модель
        prompts: Список промптов
        sampling_params_list: Список параметров для каждого промпта
        show_progress: Показывать ли прогресс
    
    Returns:
        Список сгенерированных текстов
    """
    if sampling_params_list is None:
        # Используем один набор параметров для всех
        return generate_batch(llm, prompts, show_progress=show_progress)
    
    assert len(prompts) == len(sampling_params_list), "Количество промптов и параметров должно совпадать"
    
    start_time = time.time()
    
    if show_progress:
        print(f"Генерация для {len(prompts)} промптов с индивидуальными параметрами...")
    
    # vLLM поддерживает разные параметры для разных промптов
    generated_texts = []
    
    for i, (prompt, params) in enumerate(zip(prompts, sampling_params_list)):
        outputs = llm.generate([prompt], params)
        generated_texts.append(outputs[0].outputs[0].text)
    
    elapsed_time = time.time() - start_time
    
    if show_progress:
        print(f"Генерация завершена за {elapsed_time:.2f}s")
    
    return generated_texts

# Пример: разные параметры для разных типов задач
mixed_prompts = [
    "Write a creative story about a robot:",  # Творческая задача
    "What is 2+2?",  # Точный ответ
    "Explain the theory of relativity:",  # Информативная задача
]

mixed_params = [
    create_sampling_params(temperature=1.0, max_tokens=200),  # Высокая креативность
    create_sampling_params(temperature=0.1, max_tokens=50),   # Низкая креативность
    create_sampling_params(temperature=0.5, max_tokens=300),  # Средняя креативность
]

mixed_responses = generate_batch_with_params(llm, mixed_prompts, mixed_params)

for prompt, response in zip(mixed_prompts, mixed_responses):
    print(f"\nPrompt: {prompt}")
    print(f"Response: {response[:150]}...")  # Первые 150 символов

## 7. Обработка датасета (Competition Pipeline)

In [ ]:
def process_dataset_in_batches(
    llm: LLM,
    df: pd.DataFrame,
    prompt_column: str,
    batch_size: int = 32,
    sampling_params: Optional[SamplingParams] = None,
    output_column: str = "generated_text"
) -> pd.DataFrame:
    """
    Обрабатывает датасет батчами для соревнований
    
    Args:
        llm: vLLM модель
        df: DataFrame с данными
        prompt_column: Название колонки с промптами
        batch_size: Размер батча
        sampling_params: Параметры генерации
        output_column: Название колонки для результатов
    
    Returns:
        DataFrame с добавленной колонкой результатов
    """
    if sampling_params is None:
        sampling_params = create_sampling_params()
    
    df = df.copy()
    prompts = df[prompt_column].tolist()
    all_generated = []
    
    print(f"Обработка {len(prompts)} промптов батчами по {batch_size}...")
    
    # Обработка батчами
    for i in tqdm(range(0, len(prompts), batch_size)):
        batch_prompts = prompts[i:i+batch_size]
        batch_outputs = llm.generate(batch_prompts, sampling_params)
        batch_texts = [output.outputs[0].text for output in batch_outputs]
        all_generated.extend(batch_texts)
    
    df[output_column] = all_generated
    return df

# Пример: обработка тестового датасета
# Создание примера датасета
sample_data = {
    'id': range(10),
    'prompt': [
        "Summarize: Machine learning is a subset of AI",
        "Translate to French: Hello world",
        "Question: What is Python? Answer:",
        "Complete: The capital of France is",
        "Classify sentiment: This movie is great!",
        "Extract entities: John works at Google in California",
        "Paraphrase: AI is changing the world",
        "Generate title: Article about climate change",
        "Answer: What is the speed of light?",
        "Simplify: Quantum entanglement is a phenomenon"
    ]
}

df_test = pd.DataFrame(sample_data)
print("Исходный датасет:")
print(df_test.head())

# Обработка
df_results = process_dataset_in_batches(
    llm, 
    df_test, 
    prompt_column='prompt',
    batch_size=4,
    sampling_params=sampling_params_precise
)

print("\nРезультаты:")
print(df_results[['id', 'prompt', 'generated_text']].head())

## 8. Создание промптов с шаблонами

In [ ]:
def create_prompts_from_template(
    df: pd.DataFrame,
    template: str,
    column_mapping: Dict[str, str]
) -> List[str]:
    """
    Создает промпты из шаблона и данных
    
    Args:
        df: DataFrame с данными
        template: Шаблон промпта с плейсхолдерами {column_name}
        column_mapping: Маппинг колонок DataFrame на плейсхолдеры
    
    Returns:
        Список промптов
    """
    prompts = []
    
    for idx, row in df.iterrows():
        prompt = template
        for placeholder, column in column_mapping.items():
            prompt = prompt.replace(f"{{{placeholder}}}", str(row[column]))
        prompts.append(prompt)
    
    return prompts

# Пример шаблонов для разных задач
TEMPLATES = {
    'classification': """Classify the following text into one of these categories: {categories}

Text: {text}

Category:""",
    
    'summarization': """Summarize the following text in {num_sentences} sentences:

{text}

Summary:""",
    
    'question_answering': """Context: {context}

Question: {question}

Answer:""",
    
    'translation': """Translate the following text from {source_lang} to {target_lang}:

{text}

Translation:""",
    
    'text_generation': """Complete the following text:

{text}

Continuation:"""
}

# Пример использования для классификации
classification_data = pd.DataFrame({
    'text': [
        "This product is amazing! I love it.",
        "Terrible service, would not recommend.",
        "It's okay, nothing special."
    ],
    'categories': ['positive, negative, neutral'] * 3
})

classification_prompts = create_prompts_from_template(
    classification_data,
    TEMPLATES['classification'],
    {'text': 'text', 'categories': 'categories'}
)

print("Пример промпта для классификации:")
print(classification_prompts[0])

# Генерация
classification_results = generate_batch(
    llm, 
    classification_prompts, 
    sampling_params_precise,
    show_progress=True
)

classification_data['predicted_category'] = classification_results
print("\nРезультаты классификации:")
print(classification_data)

## 9. Multiple Generations (N>1)

In [ ]:
def generate_multiple_candidates(
    llm: LLM,
    prompts: List[str],
    n: int = 3,
    sampling_params: Optional[SamplingParams] = None
) -> List[List[str]]:
    """
    Генерирует несколько кандидатов для каждого промпта
    
    Args:
        llm: vLLM модель
        prompts: Список промптов
        n: Количество кандидатов на промпт
        sampling_params: Параметры генерации
    
    Returns:
        Список списков кандидатов
    """
    if sampling_params is None:
        sampling_params = create_sampling_params(n=n, temperature=0.8)
    else:
        # Обновляем n в параметрах
        sampling_params.n = n
    
    outputs = llm.generate(prompts, sampling_params)
    
    # Группируем по промптам
    all_candidates = []
    for output in outputs:
        candidates = [gen.text for gen in output.outputs]
        all_candidates.append(candidates)
    
    return all_candidates

# Пример: генерация нескольких вариантов ответа
test_prompts = [
    "Write a tagline for an AI company:",
    "Create a catchy name for a coffee shop:"
]

candidates = generate_multiple_candidates(
    llm, 
    test_prompts, 
    n=5,
    sampling_params=create_sampling_params(temperature=0.9, max_tokens=50)
)

for i, (prompt, prompt_candidates) in enumerate(zip(test_prompts, candidates)):
    print(f"\nPrompt {i+1}: {prompt}")
    for j, candidate in enumerate(prompt_candidates):
        print(f"  Candidate {j+1}: {candidate.strip()}")

## 10. Сохранение и загрузка результатов

In [ ]:
def save_results(
    df: pd.DataFrame,
    output_path: str,
    format: str = 'csv'
):
    """
    Сохраняет результаты генерации
    
    Args:
        df: DataFrame с результатами
        output_path: Путь для сохранения
        format: Формат файла ('csv', 'json', 'parquet')
    """
    if format == 'csv':
        df.to_csv(output_path, index=False)
    elif format == 'json':
        df.to_json(output_path, orient='records', lines=True)
    elif format == 'parquet':
        df.to_parquet(output_path, index=False)
    else:
        raise ValueError(f"Неподдерживаемый формат: {format}")
    
    print(f"Результаты сохранены в {output_path}")

# Сохранение результатов
save_results(df_results, 'vllm_generation_results.csv', format='csv')

# Также можно сохранить метаданные генерации
metadata = {
    'model': MODEL_NAME,
    'timestamp': pd.Timestamp.now().isoformat(),
    'num_prompts': len(df_results),
    'sampling_params': {
        'temperature': DEFAULT_TEMPERATURE,
        'top_p': DEFAULT_TOP_P,
        'max_tokens': DEFAULT_MAX_TOKENS
    }
}

with open('vllm_generation_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("Метаданные сохранены!")

## 11. Полный Pipeline для Соревнований

In [ ]:
class VLLMCompetitionPipeline:
    """
    Полный pipeline для генерации в соревнованиях
    """
    
    def __init__(
        self,
        model_name: str = MODEL_NAME,
        max_model_len: int = MAX_MODEL_LEN,
        gpu_memory_utilization: float = GPU_MEMORY_UTILIZATION
    ):
        self.model_name = model_name
        self.llm = initialize_vllm_model(
            model_name=model_name,
            max_model_len=max_model_len,
            gpu_memory_utilization=gpu_memory_utilization
        )
        
    def prepare_prompts(
        self, 
        df: pd.DataFrame, 
        template: str,
        column_mapping: Dict[str, str]
    ) -> List[str]:
        """Подготовка промптов из датасета"""
        return create_prompts_from_template(df, template, column_mapping)
    
    def generate(
        self,
        prompts: List[str],
        batch_size: int = 32,
        sampling_params: Optional[SamplingParams] = None
    ) -> List[str]:
        """Генерация батчами"""
        if sampling_params is None:
            sampling_params = create_sampling_params()
        
        all_outputs = []
        
        for i in tqdm(range(0, len(prompts), batch_size), desc="Generating"):
            batch = prompts[i:i+batch_size]
            outputs = self.llm.generate(batch, sampling_params)
            texts = [out.outputs[0].text for out in outputs]
            all_outputs.extend(texts)
        
        return all_outputs
    
    def process_and_save(
        self,
        train_path: str,
        test_path: str,
        template: str,
        column_mapping: Dict[str, str],
        output_path: str = 'submission.csv',
        batch_size: int = 32,
        sampling_params: Optional[SamplingParams] = None
    ):
        """Полный pipeline: загрузка -> обработка -> сохранение"""
        
        # Загрузка данных
        print("Загрузка данных...")
        test_df = pd.read_csv(test_path)
        
        # Подготовка промптов
        print("Подготовка промптов...")
        prompts = self.prepare_prompts(test_df, template, column_mapping)
        
        # Генерация
        print(f"Генерация для {len(prompts)} примеров...")
        predictions = self.generate(prompts, batch_size, sampling_params)
        
        # Создание submission
        print("Создание submission файла...")
        submission_df = pd.DataFrame({
            'id': test_df['id'] if 'id' in test_df.columns else range(len(test_df)),
            'prediction': predictions
        })
        
        # Сохранение
        submission_df.to_csv(output_path, index=False)
        print(f"Submission сохранен в {output_path}")
        
        return submission_df

# Пример использования pipeline
print("\n=== Инициализация Competition Pipeline ===")
pipeline = VLLMCompetitionPipeline(model_name=MODEL_NAME)

# Пример обработки
# pipeline.process_and_save(
#     train_path='train.csv',
#     test_path='test.csv',
#     template=TEMPLATES['classification'],
#     column_mapping={'text': 'text', 'categories': 'categories'},
#     output_path='submission.csv',
#     batch_size=32,
#     sampling_params=create_sampling_params(temperature=0.3)
# )

## 12. Benchmark и Оптимизация

In [ ]:
def benchmark_generation(
    llm: LLM,
    num_prompts: int = 100,
    batch_sizes: List[int] = [1, 4, 8, 16, 32],
    prompt_length: int = 50,
    max_tokens: int = 100
) -> pd.DataFrame:
    """
    Бенчмарк производительности для разных размеров батчей
    
    Args:
        llm: vLLM модель
        num_prompts: Количество промптов для теста
        batch_sizes: Список размеров батчей для тестирования
        prompt_length: Длина промпта в словах
        max_tokens: Максимум токенов для генерации
    
    Returns:
        DataFrame с результатами бенчмарка
    """
    # Создание тестовых промптов
    test_prompts = [
        f"Generate a summary of: {' '.join(['word'] * prompt_length)}"
        for _ in range(num_prompts)
    ]
    
    sampling_params = create_sampling_params(max_tokens=max_tokens)
    results = []
    
    for batch_size in batch_sizes:
        print(f"\nТестирование batch_size={batch_size}...")
        start_time = time.time()
        
        # Генерация батчами
        for i in range(0, len(test_prompts), batch_size):
            batch = test_prompts[i:i+batch_size]
            _ = llm.generate(batch, sampling_params)
        
        elapsed = time.time() - start_time
        throughput = num_prompts / elapsed
        
        results.append({
            'batch_size': batch_size,
            'total_time': elapsed,
            'prompts_per_sec': throughput,
            'time_per_prompt': elapsed / num_prompts
        })
        
        print(f"  Время: {elapsed:.2f}s, Throughput: {throughput:.2f} prompts/s")
    
    return pd.DataFrame(results)

# Запуск бенчмарка
print("=== Benchmark Production Performance ===")
benchmark_results = benchmark_generation(
    llm, 
    num_prompts=20,  # Меньше для быстрого теста
    batch_sizes=[1, 4, 8],
    max_tokens=50
)

print("\nРезультаты бенчмарка:")
print(benchmark_results)
print(f"\nОптимальный batch_size: {benchmark_results.loc[benchmark_results['prompts_per_sec'].idxmax(), 'batch_size']}")

## 13. Советы по оптимизации

### Увеличение производительности:

1. **GPU Memory Utilization**: Увеличьте `gpu_memory_utilization` до 0.95 для максимального использования GPU

2. **Batch Size**: Используйте максимально возможный batch size (зависит от длины промптов и доступной памяти)

3. **Tensor Parallelism**: Для больших моделей используйте несколько GPU:
   ```python
   llm = LLM(model="meta-llama/Llama-2-70b-hf", tensor_parallel_size=4)
   ```

4. **Quantization**: Используйте квантизованные модели для экономии памяти:
   ```python
   llm = LLM(model="TheBloke/Llama-2-7B-GPTQ", quantization="gptq")
   ```

5. **Max Model Length**: Уменьшите если не нужен длинный контекст

### Качество генерации:

1. **Temperature**: 
   - 0.1-0.3: Для детерминированных задач (классификация, извлечение)
   - 0.7-0.9: Для творческих задач (генерация текста, диалоги)
   
2. **Top-P vs Top-K**: 
   - Top-P (nucleus sampling): Более динамичный выбор токенов
   - Top-K: Более предсказуемый результат
   
3. **Beam Search**: Используйте для задач где важна оптимальность (суммаризация, перевод)

4. **Repetition Penalty**: Увеличьте до 1.1-1.3 для борьбы с повторениями

### Экономия ресурсов:

1. Используйте меньшие модели когда возможно (Phi-2, Mistral-7B)
2. Установите разумный `max_tokens`
3. Используйте `stop` sequences для ранней остановки генерации

## Заключение

Этот notebook предоставляет полный pipeline для эффективной генерации текста с vLLM.

### Основные возможности:
- ✅ Эффективная батч-обработка
- ✅ Гибкие параметры генерации
- ✅ Шаблоны промптов
- ✅ Pipeline для соревнований
- ✅ Бенчмарки и оптимизация

### Следующие шаги:
1. Замените `MODEL_NAME` на вашу модель
2. Настройте `TEMPLATES` для вашей задачи
3. Загрузите ваши данные
4. Запустите `VLLMCompetitionPipeline.process_and_save()`
5. Создайте submission!

Удачи в соревнованиях! 🚀